In [1]:
!pip install torchmetrics
!pip install torchinfo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

 Import libraries

In [9]:
# Standard python libraries
import collections
# Scientific python
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np

# Great progress bar
import tqdm.auto as tqdm
# ML: PCA
import sklearn.decomposition
# Our DL stack
import torch
import torch.nn as nn
import torchvision
import torch.utils.data as torch_data
import torch.nn.functional as F

# DL utilities
import torchinfo
import torchmetrics

print(torch.__version__)
cuda = torch.cuda.is_available()
DEVICE = torch.device("cuda" if cuda else "cpu")
print(f"Using device: {device}")

2.5.1+cu124
Using device: cuda


# Define hyperparameters


In [5]:
NUM_EPOCHS = 5
BATCH_SIZE = 64
LEARNING_RATE = 0.001
IMG_SIZE = (28, 28)
W, H = IMG_SIZE
SEED = 42

Set random seed

In [ ]:
torch.manual_seed(SEED)
np.random.seed(SEED)

Load datasets

In [11]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data',
                                           train=True,
                                           transform=torchvision.transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.FashionMNIST(root='./data',
                                          train=False,
                                          transform=torchvision.transforms.ToTensor())

train_loader = torch_data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)

test_loader = torch_data.DataLoader(dataset=test_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=False)

# Define a CNN

In [14]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, 10)


    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)  # Flatten the output
        out = self.fc(out)
        return out

Create the model

In [15]:
model = ConvNet().to(DEVICE)
print("Model Summary:")
torchinfo.summary(model, input_size=(BATCH_SIZE, 1, W, H))

Model Summary:


Layer (type:depth-idx)                   Output Shape              Param #
ConvNet                                  [64, 10]                  --
├─Sequential: 1-1                        [64, 16, 14, 14]          --
│    └─Conv2d: 2-1                       [64, 16, 28, 28]          416
│    └─BatchNorm2d: 2-2                  [64, 16, 28, 28]          32
│    └─ReLU: 2-3                         [64, 16, 28, 28]          --
│    └─MaxPool2d: 2-4                    [64, 16, 14, 14]          --
├─Sequential: 1-2                        [64, 32, 7, 7]            --
│    └─Conv2d: 2-5                       [64, 32, 14, 14]          12,832
│    └─BatchNorm2d: 2-6                  [64, 32, 14, 14]          64
│    └─ReLU: 2-7                         [64, 32, 14, 14]          --
│    └─MaxPool2d: 2-8                    [64, 32, 7, 7]            --
├─Linear: 1-3                            [64, 10]                  15,690
Total params: 29,034
Trainable params: 29,034
Non-trainable params: 0
Total 

Our loss function and optimizer

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training loop

In [17]:
total_step = len(train_loader)
progress_bar = tqdm.tqdm(total=NUM_EPOCHS * total_step, desc="Training")
for epoch in range(NUM_EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        progress_bar.update(1)

progress_bar.close()

# Test the model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Test Accuracy: {(100 * correct / total)}')

Training:   0%|          | 0/4690 [00:00<?, ?it/s]

Accuracy of the model on the 10000 test images: 90.64 %
